In [27]:
import pandas as pd
import numpy as np
import os
from dotenv import load_dotenv
import google.generativeai as genai
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression


In [21]:
load_dotenv()
genai.configure(api_key=os.getenv("GEMINI_API_KEY"))
gemini = genai.GenerativeModel("gemini-3-flash-preview")

In [23]:
df = pd.read_csv("../data/credit_risk_dataset.csv")

X = df.drop(columns=["default_risk", "customer_financial_statement","risk_category"])
y = df["default_risk"]

X = pd.get_dummies(X, drop_first=True)

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)



In [28]:
model_lr = LogisticRegression(max_iter=1000)
model_lr.fit(X_train_scaled, y_train)

pred_lr = model_lr.predict(X_test_scaled)
prob_lr = model_lr.predict_proba(X_test_scaled)[:,1]

acc_lr = accuracy_score(y_test, pred_lr)
f1_lr = f1_score(y_test, pred_lr)
auc_lr = roc_auc_score(y_test, prob_lr)

cv_lr = cross_val_score(model_lr, X_train_scaled, y_train, cv=5, scoring="f1").mean()

importance_lr = pd.Series(model_lr.coef_[0], index=X.columns).sort_values(ascending=False).head(10)

In [31]:
print("\n--- Logistic Regression Classification Report ---")
print(classification_report(y_test, pred_lr))
print("Confusion Matrix:\n", confusion_matrix(y_test, pred_lr))


--- Logistic Regression Classification Report ---
              precision    recall  f1-score   support

           0       0.96      0.98      0.97       193
           1       0.91      0.85      0.88        47

    accuracy                           0.95       240
   macro avg       0.94      0.92      0.93       240
weighted avg       0.95      0.95      0.95       240

Confusion Matrix:
 [[189   4]
 [  7  40]]


In [29]:
print("Logistic Regression:", acc_lr, f1_lr, auc_lr, cv_lr)
print("\nTop Logistic Features:")
print(importance_lr)

Logistic Regression: 0.9541666666666667 0.8791208791208791 0.9938264799911807 0.8910249148670202

Top Logistic Features:
num_late_payments                1.753762
sentiment_Positive               1.303308
credit_utilization_ratio         1.283838
sentiment_Neutral                1.084105
employment_type_Self-Employed    0.306235
region_Suburban                  0.166749
loan_duration_months             0.153279
existing_loans_count             0.134565
education_level_Master           0.133570
region_Urban                     0.124887
dtype: float64


In [30]:
prompt = f"""
You are a financial data analyst.

We trained model for loan default prediction.

Logistic Regression:
Accuracy = {acc_lr:.3f}
F1 Score = {f1_lr:.3f}
ROC-AUC = {auc_lr:.3f}
Cross-Validation F1 = {cv_lr:.3f}
Top Features:
{importance_lr.to_string()}

Please provide:
1. Model performance summary
2. Financial insights about default risk patterns
3. Feature interpretation
4. Business recommendations
"""

response = gemini.generate_content(prompt).text

print("\nGemini AI Interpretation:\n")
print(response)


Gemini AI Interpretation:

As a financial data analyst, I have reviewed the Logistic Regression model performance and the underlying feature importance. Here is the analysis and strategic outlook based on the data provided.

---

### 1. Model Performance Summary
The model demonstrates **exceptional predictive power** and is highly reliable for deployment in a production environment.

*   **Discriminatory Power (ROC-AUC = 0.994):** An AUC of nearly 1.0 indicates that the model has a near-perfect ability to distinguish between a borrower who will default and one who will not. This suggests very little overlap between the distributions of the two classes.
*   **Balance & Reliability (F1 Score = 0.879 / CV F1 = 0.891):** The F1 score is high, meaning the model effectively balances Precision (avoiding false alarms) and Recall (catching actual defaults). The fact that the **Cross-Validation F1 (0.891)** is slightly higher than the test F1 suggests the model is stable and generalizes well to